In [3]:
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
import time
import glob

#========================================================================
# Keras 
# Corporación Favorita Grocery Sales Forecasting
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.layers import LSTM
from keras import callbacks
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import gc

from sklearn.model_selection import train_test_split

from keras import backend as K

from sklearn.metrics import mean_squared_error

def RMSE(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
    
#========================================================================
# Args 
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'column_0', 'index', 'first_active_month', 'no_out_flg']
#========================================================================

print("Preparing dataset...")
#========================================================================
# Data Load 
win_path = f'../features/4_winner/*.gz'
win_path = f'../model/LB3670_70leaves_colsam0322/*.gz'
# win_path_list = glob.glob(win_path) + glob.glob('../features/5_tmp/*.gz')
win_path_list = glob.glob(win_path)

base = utils.read_df_pkl('../input/base_term*0*')[[key, target, 'first_active_month']]
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
feature_list = utils.parallel_load_data(path_list=win_path_list)
df = pd.concat(feature_list, axis=1)
train = pd.concat([base_train, df.iloc[:len(base_train), :]], axis=1)
test = pd.concat([base_test, df.iloc[len(base_train):, :].reset_index(drop=True)], axis=1)
#========================================================================

def impute_feature(df, col):

    feature = df[col].values.astype('float32')

    inf_max = np.sort(feature)[::-1][0]
    inf_min = np.sort(feature)[0]
    
    if inf_max == np.inf:
        v_max = np.max(np.where(feature==inf_max, np.median(feature), feature))
        feature = np.where(feature==inf_max, v_max, feature)
    if inf_min == -np.inf:
        v_min = np.min(np.where(feature==inf_min, np.median(feature), feature))
        feature = np.where(feature==inf_min, v_min, feature)

    length = len(feature)
    null_len = len(feature[feature==feature])
    
    inf_max = feature.max()
    inf_min = feature.min()
    
    if length - null_len==0:
        pass
    
    else:
        if col.count('month_diff'):
            
            for val_max in np.sort(feature)[::-1]:
                if not(val_max==val_max):
                    continue
                feature = np.where(feature!=feature, val_max-1, feature)
                break
        else:
            for val_min in np.sort(feature):
                if not(val_min==val_min):
                    continue
                feature = np.where(feature!=feature, val_min-1, feature)
                break
            
    if str(type(feature)).count('series'):
        feature = feature.values
    return feature


# tmp_train = train.copy()
# tmp_test = test.copy()
#========================================================================
# 正規化の前処理(Null埋め, inf, -infの処理) 
for col in train.columns:
    if col in ignore_list: continue
        
    train[col] = impute_feature(train, col)
    test[col] = impute_feature(test, col)
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 45.96it/s]

Preparing dataset...


In [4]:
length = len(train)
for col in train.columns:
    tmp = train[col].dropna().shape[0]
    if length - tmp>0:
        print(col)

In [5]:
#========================================================================
# Make Dataset
use_cols = [col for col in train.columns if col not in ignore_list]
X_train, X_val = train_test_split(train, test_size=0.2)

df_pred = X_val[[key, target]].copy()

y_train = X_train[target].values
X_train = X_train[use_cols]
test = test[use_cols]
y_val = X_val[target].values
X_val = X_val[use_cols]

scaler = StandardScaler()
scaler.fit(pd.concat([X_train, X_val, test]))
X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
test[:] = scaler.transform(test)

X_train = X_train.as_matrix()
test = test.as_matrix()
X_val = X_val.as_matrix()
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
test = test.reshape((test.shape[0], 1, test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
#========================================================================


#========================================================================
# CVの準備
pred_list = []
score_list = []
#========================================================================
#========================================================================
# Corporación Favorita Grocery Sales Forecasting 1st Place NN Model
# https://www.kaggle.com/shixw125/1st-place-nn-model-public-0-507-private-0-513
def build_model():
    model = Sequential()
    model.add(LSTM(512, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(.2))

    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(128))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    # test
#     model.add(LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2])))
#     model.add(Dense(64))
    
    # original
    model.add(Dense(64))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(32))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(16))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(1))

    return model

def basic_build():
    models = Sequential()
    models.add(Dense(output_dim=1024, input_dim=input_d, init='lecun_uniform')) 
    models.add(Activation('relu')) 
    models.add(BatchNormalization())    
    models.add(Dropout(0.5))  
    models.add(Dense(512, activation='relu',init='lecun_uniform'))
    models.add(Activation('relu')) 
    models.add(BatchNormalization())    
    models.add(Dropout(0.4))  
    models.add(Dense(2, init='lecun_uniform'))
    models.add(Activation('softmax'))    
    opt = optimizers.Adam(lr=learning_rate)
    models.compile(loss='binary_crossentropy', optimizer=opt)

# Check
# X_train = X_train[:2000]
# y_train = y_train[:2000]
# X_val = X_val[:200]
# y_val = y_val[:200]

#========================================================================
# NN Model Setting 
# N_EPOCHS = 2000
N_EPOCHS = 10
# batch_size = 65536
batch_size = 128
learning_rate = 0.001
learning_rate = 0.0001

val_pred_list = []
test_pred_list = []

y = y_train
y_mean = y.mean()
model = build_model()
opt = optimizers.Adam(lr=learning_rate)
model.compile(loss=RMSE, optimizer=opt, metrics=[RMSE])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=0),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
    ]
# なぜか平均を引いてる？そのほうがfitするの？
# model.fit(X_train, y- y_mean, batch_size = batch_size, epochs = N_EPOCHS, verbose=2,
#            validation_data=(X_val, y_val - y_mean), callbacks=callbacks )
model.fit(X_train, y, batch_size = batch_size, epochs = N_EPOCHS, verbose=2,
           validation_data=(X_val, y_val), callbacks=callbacks )
val_pred_list.append(model.predict(X_val))
test_pred_list.append(model.predict(test))

y_pred = np.array(val_pred_list).reshape(np.array(val_pred_list).shape[1], )
df_pred['prediction'] = y_pred
pred_list.append(df_pred)

err = (y_val - y_pred)
score = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'RMSE: {score} | SUM: {err.sum()}')
score_list.append(score)
cv_score = np.mean(score_list)
#========================================================================

utils.to_pkl_gzip(obj=test_pred_list, path=f'../output/{start_time[4:11]}_elo_NN_stack_CV{score}')

Train on 161533 samples, validate on 40384 samples
Epoch 1/10
 - 26s - loss: 1.6302 - RMSE: 1.6302 - val_loss: 1.4485 - val_RMSE: 1.4485
Epoch 2/10
 - 23s - loss: 1.5230 - RMSE: 1.5230 - val_loss: 1.4287 - val_RMSE: 1.4287
Epoch 3/10
 - 23s - loss: 1.5012 - RMSE: 1.5012 - val_loss: 1.4206 - val_RMSE: 1.4206
Epoch 4/10
 - 23s - loss: 1.4906 - RMSE: 1.4906 - val_loss: 1.4171 - val_RMSE: 1.4171
Epoch 5/10
 - 23s - loss: 1.4811 - RMSE: 1.4811 - val_loss: 1.4138 - val_RMSE: 1.4138
Epoch 6/10
 - 23s - loss: 1.4754 - RMSE: 1.4754 - val_loss: 1.4129 - val_RMSE: 1.4129
Epoch 7/10
 - 23s - loss: 1.4711 - RMSE: 1.4711 - val_loss: 1.4107 - val_RMSE: 1.4107
Epoch 8/10
 - 23s - loss: 1.4674 - RMSE: 1.4674 - val_loss: 1.4099 - val_RMSE: 1.4099
Epoch 9/10
 - 23s - loss: 1.4634 - RMSE: 1.4634 - val_loss: 1.4103 - val_RMSE: 1.4103
Epoch 10/10
 - 23s - loss: 1.4606 - RMSE: 1.4606 - val_loss: 1.4100 - val_RMSE: 1.4100
RMSE: 3.6566551594853247 | SUM: -16097.1688223397


NameError: name 'start_time' is not defined